In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'1.12.0'

In [3]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
import cv2
# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.12.0'):
    raise ImportError('Please upgrade your TensorFlow installation to v1.12.*.')

In [4]:
%matplotlib notebook


In [5]:
from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util

In [6]:
PATH_TO_FROZEN_GRAPH = "/home/rithwik/Desktop/ML/project/models/research/mess_ssd_graphs/frozen_inference_graph.pb"

In [7]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [8]:
category_index = label_map_util.create_category_index_from_labelmap("object-detection.pbtxt", use_display_name=True)

In [9]:
category_index

{1: {'id': 1, 'name': 'jeera_rice'},
 2: {'id': 2, 'name': 'veg_kadhai'},
 3: {'id': 3, 'name': 'masoor_dal'},
 4: {'id': 4, 'name': 'onion'},
 5: {'id': 5, 'name': 'chapati'},
 6: {'id': 6, 'name': 'paneer_lababdar'},
 7: {'id': 7, 'name': 'lemon'},
 8: {'id': 8, 'name': 'rice'},
 9: {'id': 9, 'name': 'tomato'}}

In [10]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

In [11]:
PATH_TO_TEST_IMAGES_DIR = 'test_images/'

TEST_IMAGE_PATHS = [PATH_TO_TEST_IMAGES_DIR+i for i in os.listdir(PATH_TO_TEST_IMAGES_DIR)]# , 'images/raccoon-50.jpg']

# Size, in inches, of the output images.
TEST_IMAGE_PATHS

['test_images/IMG_20190319_124539.jpg',
 'test_images/IMG_20190319_124442.jpg',
 'test_images/IMG_20190319_124537.jpg',
 'test_images/IMG_20190319_124510.jpg',
 'test_images/IMG_20190319_124457.jpg',
 'test_images/IMG_20190319_124456.jpg',
 'test_images/IMG_20190319_131621.jpg']

In [12]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [13]:
for image_path in TEST_IMAGE_PATHS:
  image = Image.open(image_path)
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = load_image_into_numpy_array(image)
  image_np_nopy  = np.copy(image_np)

  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np, detection_graph)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=4)
  plt.figure()
  plt.imshow(image_np)
  plt.savefig("Results/"+image_path.split('/')[-1]+'.png')
